In [1]:
import tensorflow as tf
import os
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import csv

PH = 5

def readData(filePath) :
    x_data = []
    y_data = []    
    allList = []
    newPointx=[]
    newPointy=[]
    
    with open(filePath, 'r') as f:
        for line in f:
            allList.append(float(line))
    
    df = pd.Series(allList) #일차원 리스트를 pandas 데이터프레임화
    A1c = df[0:1]

    while True:
        for i in df[1:8]: 
            newPointx.append(float(i)) #데이터프레임 앞의 7개를 newPointx 리스트에 삽입
        newPointy.append(float(df[7 + (PH/5)])) #데이터프레임 그 다음(10번째)을 newPointy 리스트에 삽입
        ########## 당화혈색소 인풋데이터 추가
        newPointx.append(float(A1c))


        x_data.append(newPointx) #x_data 리스트에 newPointx 리스트를 삽입 (x_data는 array of array가 됨)
        y_data.append(newPointy) #위와 동일

        newPointx=[] #다음 반복을 위해 newPointx,y를 빈 리스트로 초기화
        newPointy=[]
        df=df.shift(-1) #데이터프레임 왼 쪽으로 1칸 쉬프트
        if(math.isnan(df[8 + (PH/5)])): #만약 8번째 데이터프레임이 NaN(Not a Number) 라면 반복 중지
            break

    #배열 갯수 보정작업
    #데이터의 개수가 항상 8의 배수가 아니기 때문에 x_data의 마지막 원소 리스트가 항상 7개가 아닐수 있고
    #y_data의 마지막 원소 리스트가 항상 1개가 아닐 수 있기 때문에 
    #빈 칸들은 0으로 채워주기 위한 작업
    if(len(x_data[-1]) != 8):
       xSize = 8-len(x_data[-1])
       for i in range(xSize):
           x_data[-1].append(0.0)
    if(len(y_data[-1])!=1):
       y_data[-1].append(0.0)  
    
    x_data=np.asarray(x_data)
    y_data=np.asarray(y_data)
    
    data = [x_data, y_data]
    return data;


filename = os.listdir("sch")

train_data_name = []
test_data_name = []

for fn in filename:
    if fn.find("test") != -1:
        test_data_name.append(fn)
    elif fn.find("train") != -1:
        train_data_name.append(fn)


total_x_data = []
total_y_data = []        
        
train_x_data = []
train_y_data = []
for fn in train_data_name:
    x,y = readData("sch/"+str(fn))
    train_x_data.append(x)
    total_x_data.append(x)
    train_y_data.append(y)
    total_y_data.append(y)
    

test_x_data = []
test_y_data = []
for fn in test_data_name:
    x,y = readData("sch/"+str(fn))
    test_x_data.append(x)
    total_x_data.append(x)
    test_y_data.append(y)
    total_y_data.append(y)

print(total_x_data[0])

[[ 178.   182.   180.  ...,  177.   176.    11.6]
 [ 182.   180.   179.  ...,  176.   175.    11.6]
 [ 180.   179.   178.  ...,  175.   174.    11.6]
 ..., 
 [ 169.   168.   168.  ...,  167.   164.    11.6]
 [ 168.   168.   168.  ...,  164.   166.    11.6]
 [ 168.   168.   167.  ...,  166.   166.    11.6]]


In [2]:
x = tf.placeholder(tf.float32, shape=[None,8], name='X')
y = tf.placeholder(tf.float32, shape=[None,1], name='Y')

w1 = tf.Variable(tf.random_uniform([8,25], minval = -1,maxval = 1,dtype = tf.float32))
b1 = tf.Variable(tf.zeros([1,25], dtype = tf.float32))

w2 = tf.Variable(tf.random_uniform([25,1], minval = -1,maxval = 1,dtype = tf.float32))
b2 = tf.Variable(tf.zeros([1,1], dtype = tf.float32))

h1 = tf.add(tf.matmul(x,w1),b1)
func = tf.add(tf.matmul(h1,w2),b2)

cost=tf.reduce_mean(tf.square(y-func))

optimizer = tf.train.AdamOptimizer(learning_rate=0.01)

train = optimizer.minimize(cost)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(len(total_x_data)):
    for j in range(25):
        sess.run(train,feed_dict={x:total_x_data[i], y:total_y_data[i]})
        

print("학습완료")
        
pid = []
for m,k in enumerate(total_x_data):
    pre = sess.run(func, feed_dict={x:k})
    pid.append(pre[0][])
    print(pre)
    print(pre[0])
    break
    
    
plt.plot(pid,color="red")
plt.plot(total_y_data,color="blue")
plt.title("data_pre")
plt.show()
        
        
    

학습완료
[[ 174.97067261]
 [ 172.5330658 ]
 [ 171.91815186]
 [ 171.93630981]
 [ 173.69381714]
 [ 171.54095459]
 [ 171.7243042 ]
 [ 173.18742371]
 [ 171.85723877]
 [ 172.75720215]
 [ 170.92666626]
 [ 168.82606506]
 [ 169.18707275]
 [ 169.09251404]
 [ 165.91578674]
 [ 165.73875427]
 [ 164.90203857]
 [ 163.94880676]
 [ 161.42982483]
 [ 154.79119873]
 [ 153.99317932]
 [ 154.79774475]
 [ 156.61566162]
 [ 157.44844055]
 [ 155.36260986]
 [ 159.3271637 ]
 [ 159.60614014]
 [ 158.94522095]
 [ 158.29467773]
 [ 156.73600769]
 [ 157.35055542]
 [ 156.59468079]
 [ 155.21827698]
 [ 154.79217529]
 [ 154.27915955]
 [ 153.51246643]
 [ 151.29951477]
 [ 150.12002563]
 [ 151.00375366]
 [ 153.39370728]
 [ 151.37141418]
 [ 152.85040283]
 [ 157.17279053]
 [ 156.22705078]
 [ 155.07131958]
 [ 154.66743469]
 [ 153.81643677]
 [ 153.25286865]
 [ 152.56254578]
 [ 152.80316162]
 [ 152.43978882]
 [ 149.72735596]
 [ 153.61730957]
 [ 153.95663452]
 [ 151.35798645]
 [ 152.31494141]
 [ 146.56835938]
 [ 147.72842407]
 [ 148.62

ValueError: setting an array element with a sequence.